In [2]:
# Class
# 1. 객체 모델링의 수단
# 2. ADT(abstract data type) - 추상 데이터 타입


##class안에 정의 되는 내용은 크게 3가지
## 상태값 저장을 위한 변수 (field(JAVA), member variable(C++), property(PYTHON)) 
## 수행하는 작업을 위한 함수(method, member function, method)
## 정의된 class 정보를 바탕으로 일정 메모리를 확보
## => 이런 확보된 메모리 공간 -> 인스턴스, 객체
##이런 객체를 생성하기 위해 생성자가 호출되어야 해요!

## 학생 3명이 있어요
## 각 학생이 가지는 정보는 국어, 영어, 수학 점수
## 평균, 총점 결과를 알고 싶어요!


class Student:

  #Constructor
  def __init__(self,s_name,s_kor,s_eng,s_math):
     #property 를 생성하려면 self 를 이용
    self.student_name = s_name
    self.kor = s_kor
    self.eng = s_eng
    self.math = s_math
    

  #method
  def get_total(self):
        self.total = self.kor+self.eng+self.math
        return self.total
    
stu1=Student("홍길동",10,20,30)
print(stu1.get_total())



60


In [5]:
## Ensamble 을 이용한 MNIST

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


##Graph 초기화
tf.reset_default_graph()



##Class Define
class CnnModel:
    #Constructor
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self.build_net()
      
    #Model graph 생성하는 method
    def build_net(self):
       # 입력 받은 이름으로 변수 명을 설정한다.
        with tf.variable_scope(self.name):

            # Boolean Tensor 생성 for dropout
            # tf.layers.dropout( training= True/Fals) True/False에 따라서 학습인지 / 예측인지 선택하게 됨
            # default = False
            self.training = tf.placeholder(tf.bool)

            # 입력 그래프 생성
            self.X = tf.placeholder(tf.float32, [None, 784])
            # 28x28x1로 사이즈 변환
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', 
                                     activation=tf.nn.relu)
            # Pooling Layer1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2,
                                            padding="SAME" )
            # Dropout Layer1
            dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=self.training)


            # Convolutional Layer2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            # Pooling Layer2
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],strides=2, padding='SAME' )
            # Dropout Layer2
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7, training=self.training)


            # Convolutional Layer3
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
            # Pooling Layer3
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], strides=2, padding='SAME')
            # Dropout Layer3
            dropout3 = tf.layers.dropout(inputs=pool3, rate=0.7, training=self.training)

            # Dense Layer4 with Relu
            flat = tf.reshape(dropout3, [-1, 128*4*4])
            dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
            # Dropout layer4
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)

            # Dense Layer5 with Relu
            dense5 = tf.layers.dense(inputs=dropout4, units=1050, activation=tf.nn.relu)
            # Dropout Layer5
            dropout5 = tf.layers.dropout(inputs=dense5, rate=0.5, training=self.training)


            # Logits layer : Final FC Layer5 Shape = (?, 625) -> 10
            self.logits = tf.layers.dense(inputs=dropout5, units=10)

        # Cost Function
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        # Test Model
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    #Model 학습
    def train_net(self,train_x_data,train_y_data,training=False):
        return self.sess.run([self.cost, self.optimizer], 
                             feed_dict={self.X:train_x_data, 
                                        self.Y:train_y_data,
                                        self.training:training})
     
   #Model의 Accuracy측정
    def get_accuracy(self,test_x_data,test_y_data,training=False):
        return self.sess.run(self.accuracy, 
                             feed_dict={self.X: test_x_data, 
                                        self.Y : test_y_data, 
                                        self.training:training})
    
   #Model의 Prediction
    def get_prediction(self,x_data,training=False):
        return self.sess.run(self.logits, feed_dict={self.X : x_data, self.training:training})
    


sess=tf.Session()
##1.Data Loading
mnist = input_data.read_data_sets("./data/mnist",
                                  one_hot=True)

# Hyper Prarameters
training_epochs = 15
batch_size = 100
learning_rate = 0.001

##2. Model 개수를 지정하고 생성
models = []
num_of_model = 2
#cnn_models = [CnnModel(sess,"Model" + str(x)) for x in range(num_of_model)]
for m in range(num_of_model):
    models.append(CnnModel(sess, "model"+str(m)))
                                                
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(cnn_models))
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # Train each model
        for m_idx, m in enumerate(cnn_models):
            c, _ = m.train_net(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch: ', '%04d' %(epoch + 1), 'Cost = ', avg_cost_list)
print('Training Finished')

                                                

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
Epoch:  0001 Cost =  [0.00454333 0.00434959 0.00752979 0.00747473 0.00657749]
Epoch:  0002 Cost =  [0.00613216 0.00803203 0.00644612 0.00441678 0.00534781]
Epoch:  0003 Cost =  [0.00636618 0.00524156 0.00784063 0.00752749 0.00811025]
Epoch:  0004 Cost =  [0.00492034 0.00675772 0.00442756 0.00744169 0.0047882 ]
Epoch:  0005 Cost =  [0.00461313 0.00641126 0.00445433 0.00743158 0.00830162]
Epoch:  0006 Cost =  [0.00733711 0.00469863 0.0056253  0.00654156 0.00367449]
Epoch:  0007 Cost =  [0.00614622 0.00533423 0.0071784  0.00425548 0.00550321]
Epoch:  0008 Cost =  [0.004383   0.00508251 0.00515018 0.00408294 0.0041665 ]
Epoch:  0009 Cost =  [0.00602476 0.00435512 0.0031377  0.00430628 0.00804729]
Epoch:  0010 Cost =  [0.00698834 0.00661546 0.00254544 0.00508232 0.00366116]
Epoch:  0011 Cost 